In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [3]:
import json
import numpy as np
import torch

def read_question_answer_file(file_path):
    """Reads a JSONL file with question-answer data and returns a list of dictionaries."""
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))  # Parse each line as JSON
    return data

# Load your dataset
dataset_path = r'med_data/phrases_no_exclude_train.jsonl' 
#dataset_path = r'C:\Users\ranad\OneDrive - University of Glasgow\Attachments\Msc Final Year project\Data\MedQA-USMLE-4-options\phrases_no_exclude_train.jsonl'  # Replace with the path to your JSON file
questions_data = read_question_answer_file(dataset_path)

In [4]:
print(questions_data[0]['question'])
print(questions_data[0]['answer_idx'])
print(questions_data[0]['options'])

A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?
D
{'A': 'Ampicillin', 'B': 'Ceftriaxone', 'C': 'Doxycycline', 'D': 'Nitrofurantoin'}


In [5]:
# !pip install wikipedia
# !pip install Wikipedia-API

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=5fad0be54585ad0bde5e23bccb7009e48ad2c2bb1cdd174520896b1f70d6a936
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [6]:
import re

def clean_text(text):
  """Removes special characters and extra whitespace from text.

  Args:
    text: The input text to be cleaned.

  Returns:
    The cleaned text.
  """

  # Remove special characters using regular expression
  text = re.sub(r"[^\w\s]", "", text)

  # Remove extra whitespace
  text = " ".join(text.split())

  return text

In [15]:
# def save_string_to_file(text, filename):
#   """Saves a string to a text file.

#   Args:
#     text: The string to be saved.
#     filename: The name of the file to create.
#   """

#   with open(filename, 'w', encoding='utf-8') as f:
#     f.write(text)

# # # Example usage:
# # my_string = "This is the text I want to save."
# # save_string_to_file(my_string, "output.txt")

def save_string_to_file(data, filename):
  """Saves a string to a text file.

  Args:
    text: The string to be saved.
    filename: The name of the file to create.
  """

  with open(filename, "w",encoding='utf-8') as f:
    json.dump(data, f)

In [8]:
def sanitize_filename(filename):
  """Sanitizes a filename by replacing special characters with underscores.

  Args:
    filename: The original filename.

  Returns:
    The sanitized filename.
  """

  # Replace non-alphanumeric characters with underscores
  filename = re.sub(r'[^\w]', '_', filename)

  # Remove leading and trailing underscores
  filename = filename.strip('_')

  return filename

In [13]:
def split_text_to_segments(text):
    # Split text by either a single newline \n or double newline \n\n
    segments = re.split(r'\n{1,2}', text)
    
    # Remove any empty strings from the list (if there are consecutive newlines)
    segments = [segment for segment in segments if segment.strip()]
    
    return segments

In [9]:
#Initialization of the wiki and the wiki-API
import wikipedia
import wikipediaapi

result = wikipedia.search('Ampicillin', results=3)

wiki_api = wikipediaapi.Wikipedia(
        user_agent='MedQA Analysis',
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
        )
# wiki_api.page(result[0]).text

In [17]:
result
split_text_to_segments(wiki_api.page(result[0]).text)

['Ampicillin is an antibiotic belonging to the aminopenicillin class of the penicillin family. The drug is used to prevent and treat a number of bacterial infections, such as respiratory tract infections, urinary tract infections, meningitis, salmonellosis, and endocarditis. It may also be used to prevent group B streptococcal infection in newborns. It is used by mouth, by injection into a muscle, or intravenously.',
 'Common side effects include rash, nausea, and diarrhea. It should not be used in people who are allergic to penicillin. Serious side effects may include Clostridium difficile colitis or anaphylaxis. While usable in those with kidney problems, the dose may need to be decreased. Its use during pregnancy and breastfeeding appears to be generally safe.',
 "Ampicillin was discovered in 1958 and came into commercial use in 1961. It is on the World Health Organization's List of Essential Medicines. The World Health Organization classifies ampicillin as critically important for 

In [19]:
from tqdm import tqdm
counter = 0
questionLoader = questions_data[0:1000]
for question in  tqdm(questionLoader):
    options = question['options']
    for key, value in options.items():
        query = value
        data_array = []
        try:
            results = wikipedia.search(query, results=5)
            for result in results:
                data_array.append(split_text_to_segments(wiki_api.page(result).text))
                #data_text += clean_text(wiki_api.page(result).text)
        except Exception as e:
            # Handle the exception here (e.g., print a message)
            print(f"Wikipedia search failed for query: {query}")
        #save_string_to_file(data_text, "C:/Users/ranad/Documents/WikiData/" + sanitize_filename(query) + ".txt")
        save_string_to_file(data_array, "Wiki_for_embeddings/" + sanitize_filename(query) + ".json")
    counter += 1
    print("Question number:" + str(counter))

  0%|          | 1/1000 [00:04<1:12:53,  4.38s/it]

Question number:1


  0%|          | 2/1000 [00:09<1:24:14,  5.06s/it]

Question number:2


  0%|          | 3/1000 [00:12<1:04:14,  3.87s/it]

...

Question number:998


100%|█████████▉| 999/1000 [1:31:43<00:04,  4.99s/it]

Question number:999


100%|██████████| 1000/1000 [1:31:48<00:00,  5.51s/it]

Question number:1000


In [32]:
!pip install metapub

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 840.2 kB/s eta 0:00:02
   ------------------- -------------------- 0.8/1.7 MB 838.9 kB/s eta 0:00:02
   ------------------- -------------------- 0.8/1.7 MB 838.9 kB/s eta 0:00:02
   ------------------------- -------------- 1.0/1.7 MB 718.9 kB/s eta 0:00:01
   ------------------------------- -------- 1.3/1.7 MB 849.7 kB/s eta 0:00:01
   ------------------------------- -------- 1.3/1.7 MB 849.7 kB/s eta 0:00:01
   ------------------------------- -------- 1.3/1.7 MB 849.7 kB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 696.0 kB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.